In [4]:
import transformers
import torch 
import gc 
from transformers.pipelines.pt_utils import KeyDataset
from torch.utils.data import DataLoader
from datasets import load_from_disk, Dataset, DatasetDict
from tokens import *

Traceback (most recent call last):
  File "/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/utils/import_utils.py", line 1778, in _get_module
    return importlib.import_module("." + module_name, self.__name__)
  File "/usr/lib64/python3.9/importlib/__init__.py", line 127, in import_module
    return _bootstrap._gcd_import(name[level:], package, level)
  File "<frozen importlib._bootstrap>", line 1030, in _gcd_import
  File "<frozen importlib._bootstrap>", line 1007, in _find_and_load
  File "<frozen importlib._bootstrap>", line 986, in _find_and_load_unlocked
  File "<frozen importlib._bootstrap>", line 680, in _load_unlocked
  File "<frozen importlib._bootstrap_external>", line 850, in exec_module
  File "<frozen importlib._bootstrap>", line 228, in _call_with_frames_removed
  File "/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/mllama/processing_mllama.py", line 31, in <module>
    from .image_processing_mllama i

In [2]:
gc.collect()
torch.cuda.empty_cache()

In [3]:
# Load the model and tokenizer with the access token
model = transformers.AutoModelForCausalLM.from_pretrained(
    model_id,
    use_auth_token=access_token,
    torch_dtype=torch.bfloat16,
    load_in_4bit=True
)
tokenizer = transformers.AutoTokenizer.from_pretrained(
    model_id,
    use_auth_token=access_token
)
# Suppose you're using a LLaMA or GPT-like tokenizer
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token
    tokenizer.pad_token_id = tokenizer.eos_token_id
model.config.pad_token_id = tokenizer.pad_token_id

# Create the pipeline with the specified model and tokenizer
pipeline = transformers.pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    truncation=True,
    max_length=1024,
    padding="longest"
)

/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/models/auto/auto_factory.py:471: FutureWarning: The `use_auth_token` argument is deprecated and will be removed in v5 of Transformers. Please use `token` instead.
  warnings.warn(
2024-12-27 22:06:46.933333: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-12-27 22:06:46.949205: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1735330006.969459 1143307 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1735330006.975304 1143307 cuda_blas.cc:1418] Unab

In [4]:
tokenizer.pad_token_id = tokenizer.eos_token_id 
inputs = tokenizer(["Today is"], return_tensors="pt")

# Example 1: Print the scores for each token generated with Greedy Search

outputs = model.generate(**inputs, max_new_tokens=100, return_dict_in_generate=True, output_scores=True)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
/home/t/tzelilai/Desktop/Thesis/venv/lib64/python3.9/site-packages/transformers/generation/utils.py:2097: UserWarning: You are calling .generate() with the `input_ids` being on a device type different than your model's device. `input_ids` is on cpu, whereas the model is on cuda. You may experience unexpected behaviors or slower generation. Please make sure that you have put `input_ids` to the correct device by calling for example input_ids = input_ids.to('cuda') before running `.generate()`.
  warnings.warn(
From v4.47 onwards, when a model cache is to be returned, `generate` will return a `Cache` instance instead by default (as opposed to the legacy tuple of tuples format). If you want to keep returning the legacy format, please set `return_legacy_cache=True`.


In [17]:
decoded_outputs = tokenizer.decode(outputs[0][0], skip_special_tokens=True)
print(decoded_outputs)

Today is the day! The day we've all been waiting for. The day the new season of The Great British Baking Show (also known as The Great British Bake Off in the UK) premieres on PBS! I'm so excited to see what this season has in store for us. Will there be any new challenges? New judges? New bakers? I can't wait to find out.

For those who may not be familiar, The Great British Baking Show is a beloved cooking competition that celebrates


In [5]:
system_prompt = """You are an AI assistant tasked with producing NEUTRAL news article based on the provided articles.
                   Take in consideration all the provided articles and write a new article as a journalist, ensuring the content
                   is unbiased, factual, and informative."""

In [6]:
def create_chat_prompt(batch):
    prompts = []
    for example in batch["content"]: 
        prompt = f"<s>[INST] <<SYS>> {system_prompt} <</SYS>> {example} [/INST]"
        # prompt = f"<|begin_of_text|><|start_header_id|>system<|end_header_id|>{system_prompt}<|eot_id|><|start_header_id|>user<|end_header_id|>{example}<|eot_id|><|start_header_id|>assistant<|end_header_id|>"
        prompts.append(prompt)

    tokenizer.pad_token = tokenizer.eos_token

    tokenized = tokenizer(
        prompts,
        truncation=True,
        #return_overflowing_tokens=True,
        max_length=8192,
        padding=True,
        return_tensors='pt'
    )
    return tokenized

@torch.no_grad
def generate_llm_output(batch):
    # batch["input_ids"] and batch["attention_mask"] have shape [batch_size, seq_len]
    device = torch.to('cuda', 0)
    # input_ids = torch.tensor(batch["input_ids"], device=device)
    # attention_mask = torch.tensor(batch["attention_mask"], device=device)
    
    # Generate output
    # Generate multiple completions
    outputs = model.generate(
        input_ids=batch["input_ids"].to(device),
        attention_mask=batch['attention_mask'].to(device),
        max_new_tokens=612,
        do_sample=True,
        use_cache=True,
        temperature=1
    )
    # print("decoding")
    # decoded_outputs = [
    #     tokenizer.decode(seq, skip_special_tokens=True) for seq in outputs
    # ]

    decoded_output = tokenizer.decode(outputs[0][0], skip_special_tokens=True)
    return {"llm_output": decoded_output}

In [7]:
str_dataset = load_from_disk('/home/t/tzelilai/Desktop/Thesis/Llama-3.2-1B/articles_dataset')

In [8]:
test_dataset = str_dataset['test']

In [7]:
test_dataset.map(create_chat_prompt, batched=True)

Dataset({
    features: ['content', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 7511
})

In [9]:
example_datset = test_dataset.select(range(10))
print(example_datset[1])

{'content': 'The mayor of San Jose, California, on Monday announced a proposal to force gun owners to carry liability insurance on their firearms, according to KTVU-TV.\n\nUnder the proposal, all gun owners in the city would have to take out liability policies that would include coverage for accidental discharges as well as for actions taken by third parties who borrow, steal, or obtain the guns through other means.\n\n"A mayor doesn\'t have the luxury of just offering \'thoughts and prayers,\'" Mayor Sam Liccardo (D) told the TV station. "We have to solve problems. While this is far from a complete solution, it is something we can do to reduce the harms of firearms, without waiting for Congress to take action."\n\nTwo victims of the recent shooting at the Gilroy Garlic Festival were children from San Jose — 13-year-old Keyla Salazar, and 6-year-old Stephen Romero.\n\nLiccardo called the idea a "harm reduction" approach meant to emulate other models already in effect.\n\n\n\n"We requir

In [10]:
key_dataset = KeyDataset(example_datset, "content")
outputs = pipeline(key_dataset, batch_size=2)

In [35]:
output = pipeline("HELLO")

KeyboardInterrupt: 

Nine months after belittling the Islamic State as a “JV” terrorist group, President Obama will devote an urgent prime-time address Wednesday to his plan for defeating the suddenly resurgent militants, a strategy that will rely heavily on U.S. partnership with an untested Iraqi government and unproven Syrian rebels.

In a speech from the White House at 9 p.m., Mr. Obama will tell Americans that the Islamic State is now a “high national security priority,” aides said. And the president will try to marshal public and international support for what would likely be a yearslong counterterrorism campaign, launched little more than a year after he declared that the large-scale fight against Islamist terrorism was all but over.

The president briefed House and Senate leaders of both parties at the White House on Tuesday, though it didn’t appear he laid out many specifics. An aide to GOP House Speaker John A. Boehner said the president went over some of the ideas he’s already floated, and a Whit

In [32]:
example_datset = example_datset.map(create_chat_prompt, batched=True)
print(example_datset)

Dataset({
    features: ['content', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [10]:
# Apply the pipeline to the dataset
def apply_pipeline(batch):
    return pipeline(batch['content'])

results = example_datset.map(apply_pipeline, batched=True, batch_size=2)

: 

In [34]:
batch_size = 2  # typically 1 if you want to handle outputs one at a time
device = torch.device('cuda', 0)
model.to(device)

You shouldn't move a model that is dispatched using accelerate hooks.


LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(128256, 4096)
    (layers): ModuleList(
      (0-31): 32 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear4bit(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear4bit(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear4bit(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear4bit(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps

In [35]:
print(example_datset)

Dataset({
    features: ['content', 'labels', 'input_ids', 'attention_mask'],
    num_rows: 10
})


In [36]:
dataset_with_llm = example_datset.map(
    generate_llm_output,
    batched=True,
    batch_size=5  # adjust as needed
)


: 